In [1]:
%load_ext autoreload
%autoreload 2
from app import db
import numpy as np

from app.models import Tournament as TournamentModel
from app.models import Bracket as BracketModel
from app.models import Match as MatchModel
from app.models import Round as RoundModel
from app.models import User as UserModel

from tournament import Tournament

import numpy as np

from sqlalchemy.sql.expression import func, select

from tournament import (
    BracketTypes,
    Tournament
)

In [2]:
db.session.close()

In [3]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [4]:
# purge all tables first

for Model in [BracketModel, TournamentModel, MatchModel, RoundModel, UserModel]:
    purge(Model)
db.session.commit()

In [5]:
# add all users to users table
users = [
    UserModel(username='TPN', email='TPN@example.com'.lower()), 
    UserModel(username='AngryFalco', email='AngryFalco@example.com'.lower()), 
    UserModel(username='Sunrisebanana', email='Sunrisebanana@example.com'.lower()), 
    UserModel(username='Ptolemy', email='Ptolemy@example.com'.lower()), 
    UserModel(username='Vik', email='Vik@example.com'.lower()), 
    UserModel(username='Kevin', email='Kevin@example.com'.lower()), 
    UserModel(username='Spaceghost', email='Spaceghost@example.com'.lower()), 
    UserModel(username='Burnaby', email='Burnaby@example.com'.lower()), 
]
[db.session.add(u) for u in users]
db.session.commit()

In [6]:
# Fall Charity LAN 2018 Melee Singles users
N_COMPETITORS = 8
seeds = [i+1 for i in range(N_COMPETITORS)]
usernames = [
    'TPN',
    'AngryFalco',
    'Sunrisebanana',
    'Ptolemy',
    'Vik',
    'Kevin',
    'Spaceghost',
    'Burnaby',
]
np.random.shuffle(usernames)
np.random.shuffle(seeds)

In [7]:
tuple_list = [(usernames[i], seeds[i]) for i in range(N_COMPETITORS)]

In [8]:
btype = BracketTypes.DOUBLE_ELIMINATION
t = Tournament(tuple_list, btype)

In [9]:
tournament_name = 'Fall Charity LAN 2018 Melee Singles'
TO = t.bracket.entrants[0]

t.post_to_db(tournament_name, TO,)

In [10]:
# t.bracket.post_to_db()

In [11]:
rds = t.bracket.rounds

In [12]:
for r in t.bracket.rounds:
    print(r)

In [13]:
r = rds[0]

In [16]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
for r in t.bracket.rounds:
    pp.pprint(vars(r),)
    print()
    for m in r.matches:
        pp.pprint(f'\t {vars(m)}')
        print('\t')
        break
    break
        

{   'bracket': <tournament.Bracket object at 0x7fca7cc34c10>,
    'isWinners': True,
    'matches': [   <tournament.Match object at 0x7fca7cc34890>,
    'numMatches': 4,
    'number': 1}

("\t {'entrant1': 'Spaceghost', 'entrant2': 'TPN', 'matchRound': "
 "<tournament.Round object at 0x7fca7cc346d0>, 'winnerPlaysInMatch': "
 "<tournament.Match object at 0x7fca7cc34410>, 'loserPlaysInMatch': "
 '<tournament.Match object at 0x7fca7cc82f10>}')
	


In [17]:
u1_query = UserModel.query.filter_by(username='AngryFalc').first()

u1 = u1_query.id if u1_query is not None else None
u1

In [18]:
UserModel.query.filter_by(username='AngryFalco').first_or_404().id

2

In [19]:
UserModel.query.filter_by(username='AngryFalco').first().id

2

In [20]:
r = \
    RoundModel(
        number=1,
        winners=True,
        bracket=b,
#         matches=matches
    )
r

NameError: name 'b' is not defined

In [21]:
db.session.add(r)
db.session.commit()

UnmappedInstanceError: Class 'tournament.Round' is not mapped

In [22]:
r.id

AttributeError: 'Round' object has no attribute 'id'

In [23]:
matches = \
    [Match(
#         users=list((usrs[i], usrs[i+1])),
        user_1 = usrs[i].id,
        user_2 = usrs[i+1].id,
        round_id=r.id
    ) for i in range(0, len(usrs), 2)]
m = matches[0]
m

NameError: name 'usrs' is not defined

In [24]:
for m in matches:
    db.session.add(m)

NameError: name 'matches' is not defined

In [25]:
db.session.commit()

In [26]:
import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)
df

,id,username,email,password_hash,about_me,last_seen
0,1,TPN,tpn@example.com,None,None,2020-02-19 01:01:29.854339
1,2,AngryFalco,angryfalco@example.com,None,None,2020-02-19 01:01:29.854744
2,3,Sunrisebanana,sunrisebanana@example.com,None,None,2020-02-19 01:01:29.854850
3,4,Ptolemy,ptolemy@example.com,None,None,2020-02-19 01:01:29.854926
4,5,Vik,vik@example.com,None,None,2020-02-19 01:01:29.854999
5,6,Kevin,kevin@example.com,None,None,2020-02-19 01:01:29.855067
6,7,Spaceghost,spaceghost@example.com,None,None,2020-02-19 01:01:29.855135
7,8,Burnaby,burnaby@example.com,None,None,2020-02-19 01:01:29.855200


In [27]:
pd.read_sql_query('select * from tournament', cnx)

,id,n_entrants,name,organizer_id
0,1,8,Fall Charity LAN 2018 Melee Singles,7


In [28]:
pd.read_sql_query('select * from bracket', cnx)

,id,bracket_type,tournament_id
0,1,DOUBLE_ELIMINATION,1


In [29]:
pd.read_sql_query('select * from bracket_entrants', cnx)

,user_id,bracket_id
0,7,1
1,8,1
2,6,1
3,4,1
4,5,1
5,2,1
6,3,1
7,1,1


In [30]:
pd.read_sql_query('select * from round', cnx)

,id,number,winners,bracket_id
0,1,1,1,1
1,2,2,1,1
2,3,3,1,1
3,4,4,1,1
4,5,5,1,1
5,6,1,0,1
6,7,2,0,1
7,8,3,0,1
8,9,4,0,1


In [31]:
pd.read_sql_query('select * from match', cnx)

,id,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to
0,1,7.0,None,1.0,None,None,1,None,None
1,2,8.0,None,3.0,None,None,1,None,None
2,3,6.0,None,2.0,None,None,1,None,None
3,4,4.0,None,5.0,None,None,1,None,None
4,5,NaN,None,NaN,None,None,2,None,None
5,6,NaN,None,NaN,None,None,2,None,None
6,7,NaN,None,NaN,None,None,3,None,None
7,8,NaN,None,NaN,None,None,4,None,None
8,9,NaN,None,NaN,None,None,5,None,None
9,10,NaN,None,NaN,None,None,6,None,None


In [25]:
# create some new matches for winners/losers to advance to
matches = \
    [Match(
#         users=list((usrs[i], usrs[i+1])),
#         user_1 = usrs[i].id,
#         user_2 = usrs[i+1].id,
        round_id=r.id
    ) for i in range(2)]
m = matches[0]

In [26]:
db.session.add(m)
db.session.commit()

In [27]:
m

<match 5 between None and None>